Import all  important laibraries  and install them in colab run time 

In [ ]:
!pip install transformers[torch]
import torch
!pip install sentencepiece
!pip install accelerate

In [ ]:
import torch

In [ ]:
# !git clone https://huggingface.co/decapoda-research/llama-13b-hf

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install accelerate

Import llama tokennizor from transformers to get tokenizor from pretrained model of llama

In [ ]:
from transformers import LlamaTokenizer
tokonizor = LlamaTokenizer.from_pretrained('aleksickx/llama-7b-hf')

With the help of accelarator library we are creating device map which help  us to run large model like llama to run on single gpu 

In [3]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("aleksickx/llama-7b-hf")
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

device_map = infer_auto_device_map(model)

device map is basically distrubution of weights of  model on system ram , gpu and  disk 

In [4]:
device_map = infer_auto_device_map(model, no_split_module_classes=["OPTDecoderLayer"])

In [ ]:
device_map

download pretrained model from hugging face

In [6]:
import torch
from transformers import AutoModelForCausalLM

checkpoint = "aleksickx/llama-7b-hf"
device_map["model.layers.28"] = "disk"
model = AutoModelForCausalLM.from_pretrained(
    checkpoint, device_map=device_map, offload_folder="offload", offload_state_dict = True, torch_dtype=torch.float16
)

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:
from transformers import LlamaTokenizer

generate response on the given promt 

In [11]:
inputs = tokonizor("what is happiness ?", return_tensors="pt")
inputs = inputs.to(0)
output = model.generate(inputs["input_ids"],max_new_tokens=30)
print(tokonizor.decode(output[0].tolist()),end='\n')
# print(type(tokonizor.decode(output[0].tolist())))


 ⁇  what is happiness ?
The answer is simple. Happiness is a state of mind. It is a state of mind that is free from worries, free from


In [ ]:
from transformers import pipeline

# Load the GPT-J-6B model
# model_name = "EleutherAI/gpt-j-6B"
chatbot = pipeline("text-generation", model=model, tokenizer=tokonizor)

# Define a function to chat with the chatbot
def chat_with_bot():
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Goodbye!")
            break
        chatbot_output = chatbot(user_input, max_length=50, do_sample=True)
        bot_response = chatbot_output[0]["generated_text"].strip()
        print("Chatbot:", bot_response)

# Start the chatbot
# chat_with_bot()


In [ ]:
chat_with_bot()